In [1]:
import glob
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow import keras
from keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2024-03-11 18:25:02.087827: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-11 18:25:03.245276: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


2024-03-11 18:25:04.030745: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-11 18:25:04.185163: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [3]:
dir_path = "./archive/chest_xray/chest_xray"

In [4]:
''' training path '''
train_p = os.path.join(dir_path, "train")

''' test path '''
test_p =os.path.join(dir_path, "test")

''' val path '''
val_p = os.path.join(dir_path, "val")

In [5]:
''' PNEUMONIA images '''
p_train_images = glob.glob(train_p + "/PNEUMONIA/*.jpeg")

''' NORMAL  images '''
n_train_images = glob.glob(train_p + "/NORMAL/*.jpeg")

In [6]:
df = pd.DataFrame(np.concatenate([[0]*len(n_train_images) , [1] *  len(p_train_images)]),columns=["class"])

In [7]:
sns.countplot(x = "Label", data=df)

" sns.countplot(df['class'],data=df) "

In [8]:
''' Data Augmentation '''
train_ds = ImageDataGenerator(rescale =1/255, shear_range=10, zoom_range = 0.2, horizontal_flip = True, 
                              width_shift_range=0.2, fill_mode = 'nearest')

val_datagen = ImageDataGenerator(
    rescale = 1/255
)

In [9]:
''' VGG16 Model '''

model1 = VGG16(include_top = False,weights='imagenet')

''' freezing layers '''
for i in model1.layers:
    i.trainable=False
    
out = model1.output

''' GlobalAveragePooling Layer '''
out = keras.layers.GlobalAveragePooling2D()(out)
out = keras.layers.Dense(128, activation='relu')(out)

''' Classification Layer '''
pred = keras.layers.Dense(1, activation='sigmoid')(out)

''' final model '''
model= keras.Model(model1.input, pred)

lr = 0.1
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(lr, decay_steps=100000, decay_rate=0.96, staircase=True)

''' Compile the model '''
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.RMSprop(lr_schedule), metrics=['accuracy'])

In [10]:
''' lets see what the model looks like '''
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, None, None, 3)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, None, None, 64) │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, None, None, 64) │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, None, None, 64) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, None, None,     │        73,856 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, None, None,     │       147,584 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, None, None,     │             0 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, None, None,     │       295,168 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, None, None,     │       590,080 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, None, None,     │       590,080 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, None, None,     │             0 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, None, None,     │     1,180,160 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, None, None,     │             0 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │             

 Total params: 14,780,481 (56.38 MB)

 Trainable params: 65,793 (257.00 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [11]:
''' flow from directory '''
train_ds = train_ds.flow_from_directory(train_p, target_size=(150,150), batch_size= 16, class_mode='binary')

val_ds = val_datagen.flow_from_directory(val_p, target_size=(150,150),batch_size=8, class_mode='binary')

test_ds = val_datagen.flow_from_directory(test_p, target_size=(150,150), batch_size=32, class_mode='binary')

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [12]:
''' training '''
history = model.fit(train_ds, steps_per_epoch = 100, epochs=20, validation_data=val_ds)

Epoch 1/20


/home/mohamed/Desktop/Projects/Mini-Projects-Python/Machine Learning/.venv/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


100/100 ━━━━━━━━━━━━━━━━━━━━ 54s 519ms/step - accuracy: 0.7443 - loss: 8.1113 - val_accuracy: 0.8125 - val_loss: 0.4679
Epoch 2/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 51s 514ms/step - accuracy: 0.8225 - loss: 0.3833 - val_accuracy: 0.8750 - val_loss: 0.3873
Epoch 3/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 51s 507ms/step - accuracy: 0.8577 - loss: 0.3356 - val_accuracy: 0.7500 - val_loss: 0.6080
Epoch 4/20
 26/100 ━━━━━━━━━━━━━━━━━━━━ 36s 497ms/step - accuracy: 0.8840 - loss: 0.2477

2024-03-11 18:27:54.046685: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


100/100 ━━━━━━━━━━━━━━━━━━━━ 14s 131ms/step - accuracy: 0.8862 - loss: 0.2552 - val_accuracy: 0.8750 - val_loss: 0.3094
Epoch 5/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 52s 509ms/step - accuracy: 0.8820 - loss: 0.2941 - val_accuracy: 0.8750 - val_loss: 0.2594
Epoch 6/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 51s 507ms/step - accuracy: 0.8943 - loss: 0.2733 - val_accuracy: 0.9375 - val_loss: 0.3045
Epoch 7/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 51s 513ms/step - accuracy: 0.8840 - loss: 0.3126 - val_accuracy: 0.9375 - val_loss: 0.2746
Epoch 8/20
 26/100 ━━━━━━━━━━━━━━━━━━━━ 38s 514ms/step - accuracy: 0.9105 - loss: 0.2580

...

100/100 ━━━━━━━━━━━━━━━━━━━━ 13s 131ms/step - accuracy: 0.9358 - loss: 0.1629 - val_accuracy: 0.6875 - val_loss: 0.7910
Epoch 13/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 51s 501ms/step - accuracy: 0.9105 - loss: 0.2410 - val_accuracy: 0.7500 - val_loss: 0.3642
Epoch 14/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 50s 502ms/step - accuracy: 0.9014 - loss: 0.2406 - val_accuracy: 0.6875 - val_loss: 0.4873
Epoch 15/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 50s 500ms/step - accuracy: 0.9143 - loss: 0.2221 - val_accuracy: 0.8125 - val_loss: 0.3970
Epoch 16/20
 26/100 ━━━━━━━━━━━━━━━━━━━━ 36s 495ms/step - accuracy: 0.9321 - loss: 0.2107

2024-03-11 18:36:12.558651: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


100/100 ━━━━━━━━━━━━━━━━━━━━ 13s 131ms/step - accuracy: 0.9219 - loss: 0.2220 - val_accuracy: 0.7500 - val_loss: 0.5119
Epoch 17/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 52s 504ms/step - accuracy: 0.9154 - loss: 0.2395 - val_accuracy: 0.8750 - val_loss: 0.2908
Epoch 18/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 50s 501ms/step - accuracy: 0.8982 - loss: 0.2499 - val_accuracy: 0.9375 - val_loss: 0.1321
Epoch 19/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 51s 505ms/step - accuracy: 0.9092 - loss: 0.2398 - val_accuracy: 0.8750 - val_loss: 0.2490
Epoch 20/20
 26/100 ━━━━━━━━━━━━━━━━━━━━ 37s 501ms/step - accuracy: 0.8979 - loss: 0.2965

2024-03-11 18:38:58.263708: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


100/100 ━━━━━━━━━━━━━━━━━━━━ 14s 132ms/step - accuracy: 0.9130 - loss: 0.2450 - val_accuracy: 0.9375 - val_loss: 0.1510


In [13]:
''' evaluating model '''
accuracy = model.evaluate(test_ds)[1]

20/20 ━━━━━━━━━━━━━━━━━━━━ 20s 964ms/step - accuracy: 0.9191 - loss: 0.2298


In [14]:
accuracy

0.9054487347602844

In [15]:
''' unfreezing layers '''
for layer in model1.layers:
    layer.trainable = True

In [16]:
'''Finetuning the model'''
lr = 1e-5
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(lr, decay_steps=100000, decay_rate=0.96, staircase=True)

''' compile the model '''
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr_schedule), loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[keras.metrics.BinaryAccuracy()])

In [18]:
''' training '''
model.fit(train_ds, steps_per_epoch = 50, epochs=20, validation_data=val_ds)

Epoch 1/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 112s 2s/step - binary_accuracy: 0.9641 - loss: 0.1187 - val_binary_accuracy: 0.6875 - val_loss: 0.7510
Epoch 2/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 111s 2s/step - binary_accuracy: 0.9730 - loss: 0.1153 - val_binary_accuracy: 0.7500 - val_loss: 0.4623
Epoch 3/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 111s 2s/step - binary_accuracy: 0.9633 - loss: 0.1099 - val_binary_accuracy: 0.8750 - val_loss: 0.2271
Epoch 4/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 111s 2s/step - binary_accuracy: 0.9585 - loss: 0.1143 - val_binary_accuracy: 0.7500 - val_loss: 0.7671
Epoch 5/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 111s 2s/step - binary_accuracy: 0.9658 - loss: 0.1144 - val_binary_accuracy: 0.7500 - val_loss: 0.5481
Epoch 6/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 111s 2s/step - binary_accuracy: 0.9722 - loss: 0.0881 - val_binary_accuracy: 0.6875 - val_loss: 0.9724
Epoch 7/20
26/50 ━━━━━━━━━━━━━━━━━━━━ 52s 2s/step - binary_accuracy: 0.9456 - loss: 0.1412

2024-03-11 19:22:06.672496: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


50/50 ━━━━━━━━━━━━━━━━━━━━ 58s 1s/step - binary_accuracy: 0.9452 - loss: 0.1411 - val_binary_accuracy: 0.8125 - val_loss: 0.3673
Epoch 8/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 113s 2s/step - binary_accuracy: 0.9593 - loss: 0.1179 - val_binary_accuracy: 0.8750 - val_loss: 0.3027
Epoch 9/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 119s 2s/step - binary_accuracy: 0.9625 - loss: 0.1035 - val_binary_accuracy: 0.9375 - val_loss: 0.2166
Epoch 10/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 122s 2s/step - binary_accuracy: 0.9713 - loss: 0.1244 - val_binary_accuracy: 0.6250 - val_loss: 1.3137
Epoch 11/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 122s 2s/step - binary_accuracy: 0.9753 - loss: 0.0792 - val_binary_accuracy: 0.8125 - val_loss: 0.3923
Epoch 12/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 122s 2s/step - binary_accuracy: 0.9597 - loss: 0.1318 - val_binary_accuracy: 0.8750 - val_loss: 0.2013
Epoch 13/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 122s 2s/step - binary_accuracy: 0.9607 - loss: 0.1217 - val_binary_accuracy: 1.0000 - val_loss: 0.1563
Epoch 14/20
26/50 ━━━

2024-03-11 19:35:10.869435: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


50/50 ━━━━━━━━━━━━━━━━━━━━ 64s 1s/step - binary_accuracy: 0.9660 - loss: 0.1148 - val_binary_accuracy: 0.7500 - val_loss: 0.5638
Epoch 15/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 131s 3s/step - binary_accuracy: 0.9649 - loss: 0.1044 - val_binary_accuracy: 0.6875 - val_loss: 0.7332
Epoch 16/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 116s 2s/step - binary_accuracy: 0.9662 - loss: 0.0915 - val_binary_accuracy: 0.6875 - val_loss: 1.4586
Epoch 17/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 110s 2s/step - binary_accuracy: 0.9661 - loss: 0.0777 - val_binary_accuracy: 0.6875 - val_loss: 1.0701
Epoch 18/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 117s 2s/step - binary_accuracy: 0.9672 - loss: 0.1126 - val_binary_accuracy: 0.7500 - val_loss: 0.8008
Epoch 19/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 105s 2s/step - binary_accuracy: 0.9763 - loss: 0.0912 - val_binary_accuracy: 0.8125 - val_loss: 0.2603
Epoch 20/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 111s 2s/step - binary_accuracy: 0.9708 - loss: 0.0966 - val_binary_accuracy: 0.9375 - val_loss: 0.1732


In [19]:
accuracy = model.evaluate(test_ds)[1]
accuracy

20/20 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - binary_accuracy: 0.9194 - loss: 0.2196


0.9182692170143127